<a href="https://colab.research.google.com/github/wilsoncwc/dontpatronizeme/blob/main/NLP_coursework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Helpful Documentation Links

Huggingface https://huggingface.co/docs/transformers/v4.16.2/en/index

Simple Transformers https://simpletransformers.ai/docs/classification-models/

Pandas https://pandas.pydata.org/docs/reference/frame.html




# Main imports and code

In [1]:
# check which gpu we're using
!nvidia-smi

Wed Feb 16 00:01:44 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# No need to restart runtime if not plotting graphs
!pip install transformers simpletransformers tensorboardx nlpaug matplotlib==3.4.0 unidecode autocorrect contractions

from simpletransformers.classification import ClassificationModel, ClassificationArgs, MultiLabelClassificationModel, MultiLabelClassificationArgs
import pandas as pd
import logging
import os
import torch
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
import spacy

# print the entire text
pd.set_option('display.max_colwidth', None)

# prepare logger
logging.basicConfig(level=logging.INFO)

transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

# check gpu
cuda_available = torch.cuda.is_available()

print('Cuda available? ',cuda_available)

if cuda_available:
  import tensorflow as tf
  # Get the GPU device name.
  device_name = tf.test.gpu_device_name()
  # The device name should look like the following:
  if device_name == '/device:GPU:0':
      print('Found GPU at: {}'.format(device_name))
  else:
      raise SystemError('GPU device not found')

# Fetch Don't Patronize Me! data (one cell!)

Edits:

- (15 Feb) Loading from Drive instead of Github now

In [3]:
from dont_patronize_me import DontPatronizeMe

def labels2file(p, outf_path):
	# save predictions to an output file
	with open(outf_path,'w') as outf:
		for pi in p:
			outf.write(','.join([str(k) for k in pi])+'\n')

dpm = DontPatronizeMe('.', '.')
dpm.load_task1()

# Load paragraph ids
trids = pd.read_csv('train_semeval_parids-labels.csv')
teids = pd.read_csv('dev_semeval_parids-labels.csv')
print(trids.head())

trids.par_id = trids.par_id.astype(str)
teids.par_id = teids.par_id.astype(str)

Mounted at /content/drive
/content/drive/MyDrive/NLP CW
Map of label to numerical label:
{'Unbalanced_power_relations': 0, 'Shallow_solution': 1, 'Presupposition': 2, 'Authority_voice': 3, 'Metaphors': 4, 'Compassion': 5, 'The_poorer_the_merrier': 6}
   par_id                  label
0    4341  [1, 0, 0, 1, 0, 0, 0]
1    4136  [0, 1, 0, 0, 0, 0, 0]
2   10352  [1, 0, 0, 0, 0, 1, 0]
3    8279  [0, 0, 0, 1, 0, 0, 0]
4    1164  [1, 0, 0, 1, 1, 1, 0]


# Rebuild training & test sets

In [5]:
def rebuild_training_set(dpm):
    rows = [] # will contain par_id, label and text
    for idx in range(len(trids)):  
        parid = trids.par_id[idx]
        #print(parid)
        # select row from original dataset to retrieve `text` and binary label
        text = dpm.train_task1_df.loc[dpm.train_task1_df.par_id == parid].text.values[0]
        label = dpm.train_task1_df.loc[dpm.train_task1_df.par_id == parid].label.values[0]
        rows.append({
            'par_id':parid,
            'text':text,
            'label':label
        })

    print(f'Loaded {len(rows)} paragraphs to train set')
    return pd.DataFrame(rows)

def rebuild_test_set(dpm):
    rows = [] # will contain par_id, label and text
    for idx in range(len(teids)):  
        parid = teids.par_id[idx]
        #print(parid)
        # select row from original dataset
        text = dpm.train_task1_df.loc[dpm.train_task1_df.par_id == parid].text.values[0]
        label = dpm.train_task1_df.loc[dpm.train_task1_df.par_id == parid].label.values[0]
        rows.append({
            'par_id':parid,
            'text':text,
            'label':label
        })
    print(f'Loaded {len(rows)} paragraphs to test set')
    return pd.DataFrame(rows)

In [6]:
trdf1 = rebuild_training_set(dpm)
tedf1 = rebuild_test_set(dpm)

Loaded 8375 paragraphs to train set
Loaded 2094 paragraphs to test set


# Data Analysis

Found this cheatsheet useful: https://raw.githubusercontent.com/pandas-dev/pandas/main/doc/cheatsheet/Pandas_Cheat_Sheet.pdf

In [ ]:
from transformers import AutoTokenizer

base_model = "roberta-base"

tokenizer = AutoTokenizer.from_pretrained(base_model)
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

# def tokenizer(text): 
#   doc = spacy_nlp(text)
#   # Remove stop words, punctuation symbols and non alphabetic characters
#   tokens = [token.text.lower() for token in doc if not token.is_stop 
#             and not token.is_punct
#             and token.is_alpha] #keep only alphabetic characters
#   return tokens
tokenizer.tokenize("I have a new GPU!")


Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

['I', 'Ġhave', 'Ġa', 'Ġnew', 'ĠGPU', '!']

In [ ]:
df = dpm.train_task1_df
print(f'Number of paragraphs: {len(df)}')
positive_df = df.query("label == 1")
print(f'Number of positive paragraphs: {len(positive_df)}')
print(f'Number of negative paragraphs: {len(df) - len(positive_df)}')
print(f'Columns {df.columns}')

def plot_stacked_counts(data, group, title=""):
  counts = data.groupby([group, 'label']).size().unstack()
  counts['sum'] = counts[1] + counts[0]
  counts = counts.sort_values(by='sum')
  counts.drop('sum', axis=1)
  counts = counts[[1,0]]
  percentages = round(100 * counts[0] / (counts[1] + counts[0]), 2)
  percent_strs = [f'{percent}%' for percent in percentages]
  ax = counts.plot.barh(stacked=True, figsize=(10,10))
  if title:
    plt.title(title, fontsize=16)
    # for container in ax.containers:
    #   ax.bar_label(container)
    ax.bar_label(ax.containers[1], labels=percent_strs, label_type='center')
    plt.ylabel('Keyword', fontsize=12)
    plt.xlabel('Paragraph count', fontsize=12)
    plt.legend(title='Label', fontsize=12, loc='lower right')
    # plt.savefig(f'{group}-stacked-counts')
  plt.show()

def plot_analysis_graphs():
  plot_stacked_counts(df, 'keyword', title='Number of paragraphs in the dataset by keyword and label')
  # plot_stacked_counts(df, 'country')

  count_chars = (lambda par: len(par))
  count_tokens = (lambda par: len(tokenizer.tokenize(par)))

  df['token_count'] = df['text'].apply(count_tokens)
  # df.boxplot('token_count', by=['keyword', 'label'], figsize=(30,10))
  # plt.yscale('log')
  # plt.show()

  df.boxplot('token_count', by=['keyword'], figsize=(15,10))
  plt.suptitle('')
  plt.yscale('log')
  plt.ylabel('Token count (log scale)', fontsize=12)
  plt.xlabel('Keyword', fontsize=12)
  plt.title('Distribution of token counts by keyword', fontsize=16)
  plt.savefig('token-count-keyword')
  plt.show()


  # df.boxplot('token_count', by=['orig_label'], figsize=(15,10))
  # plt.yscale('log')
  # plt.show()

# plot_analysis_graphs()

Number of paragraphs: 10469
Number of positive paragraphs: 993
Number of negative paragraphs: 9476
Columns Index(['par_id', 'art_id', 'keyword', 'country', 'text', 'label',
       'orig_label'],
      dtype='object')


# Data Preprocessing

- [Useful guide on Kaggle](https://www.kaggle.com/techykajal/text-pre-processing/notebook)
- [SpaCy's stop words](https://github.com/explosion/spaCy/blob/master/spacy/lang/en/stop_words.py)
- [Why PyContractions](https://medium.com/@lukei_3514/dealing-with-contractions-in-nlp-d6174300876b) -- but then it can't be installed on colab bc of a bug in the `language-check` dependency

In [20]:
import re
import string
import contractions

from bs4 import BeautifulSoup
from unidecode import unidecode
from autocorrect import Speller
# from pycontractions import Contractions
from spacy.lang.en import English
from spacy.lang.en.stop_words import STOP_WORDS

nlp = English()
spell = Speller(lang='en')

""" 
    Function naming convention
    => 'remove' means that original token gets purged completely
    => 'replace' means that it gets substituted with something, e.g. ' '
"""

def replace_contractions(text):
    """ Contraction expansion (e.g. isn't => is not) """
    return " ".join([contractions.fix(word) for word in text.split()] )

def remove_numbers(text):
    text = re.sub(r'[0-9]+', ' ', text)
    return text

def remove_stop_words(text):
    """ Removes stop words using spacy """
    return " ".join(token.lemma_ for token in nlp(text) if not token.is_stop)


def remove_html(text):
    soup = BeautifulSoup(text, "html.parser")
    # Extract non html text
    stripped_text = soup.get_text(separator=" ")
    return stripped_text


def remove_links(text):
    # Remove all links starting with http(s)
    text = re.sub(r'http\S+', '', text)
    # Remove all links ending with .com
    text = re.sub(r"\ [A-Za-z]*\.com", " ", text)
    return text


def remove_extra_whitespace(text):
    pattern = re.compile(r'\s+') 
    text = re.sub(pattern, ' ', text)
    # For instances where there is no space after '?' and ')'
    text = text.replace('?', '? ').replace(')', ') ')
    return text


def replace_newlines_tabs(text):
    text = text.replace('\\n', ' ').replace('\n', ' ').replace('\t',' ').replace('\\', ' ')
    return text


def replace_accented_chars(text):
    text = unidecode(text)
    return text


def reduce_character_repetition(text):
    # Limit all character repetition to two
    chars = re.compile(r'([A-Za-z])\1{1,}', re.DOTALL)
    text = chars.sub(r'\1\1', text) 
    
    # Limit all punctuation repetition to one
    puncts = re.compile(r'([.,/#!$%^&*?;:{}=_`~()+-])\1{1,}')
    text = puncts.sub(r'\1', text)

    return text

def replace_special_chars(text):
    """ Regex: Replace everything except those in the square brackets with whitespace
        ,.?!:$-% are considered important by the original code author"""
    text = re.sub(r'[^a-zA-Z0-9:;$-,%.?!]+', ' ', text)
    return text


def fix_spelling(text):
    text = spell(text)
    return text

def preprocessing(text, accented_chars=True, contractions=True, lemma=True,
                  extra_whitespace=True, newlines_tabs=True, repetition=True, 
                  lowercase=True, punctuation=True, spelling=True,
                  html=True, links=True, special_chars=True, stop_words=True,
                  numbers=True):
    """Master function to apply preprocessing methods to text
       Order of application matters! """
    
    if lowercase:
        text = text.lower()

    if newlines_tabs:
        text = replace_newlines_tabs(text)

    if html:
        text = remove_html(text)
    
    if links:
        text = remove_links(text)
    
    if accented_chars:
        text = replace_accented_chars(text)
    
    if repetition:
        text = reduce_character_repetition(text)

    if special_chars:
        text = replace_special_chars(text)

    if numbers:
        text = remove_numbers(text)

    if extra_whitespace:
        text = remove_extra_whitespace(text)

    # if spelling:
    #     text = fix_spelling(text)

    if stop_words:
        text = remove_stop_words(text)

    # if lemma
    #     text = lemmatize(text)

    if contractions:
        text = replace_contractions(text)

    return text

def get_cleaned_dataset(df):
    """Return a new dataset with """
    new_df = df.copy()
    new_df['text'] = new_df['clean_text']
    print(new_df.head())
    return new_df

In [21]:
trdf1['clean_text'] = trdf1['text'].apply(preprocessing)
tedf1['clean_text'] = tedf1['text'].apply(preprocessing)

In [22]:
# compare original and preprocessed text for both labels
pcl = trdf1.loc[trdf1['label'] == 1].sample(10)
non_pcl = trdf1.loc[trdf1['label'] == 0].sample(10)
pd.concat([pcl, non_pcl])[['text', 'clean_text', 'label']]

,text,clean_text,label
86,If these problems are not tackled there is every chance that another reporter will be back in another 20 years wondering how it is possible that vulnerable children are still being cursed with hunger and a horribly early death .,problems tackled chance reporter years wondering possible vulnerable children cursed hunger horribly early death .,1
354,Bernie is asking other boutique owners to follow her lead and offer up dresses to those in need .,bernie asking boutique owners follow lead offer dresses need .,1
436,"What is your advice to Nigerian women ? My advice to women is that they should step forward and be bold , as stated in the UN declaration . I want women to be encouraged . Pick up something that you can advocate for . It is not all about money . It is about the people you are surrounded with . Start from your immediate community and you will get to where you want to go","advice nigerian women ? advice women step forward bold , stated un declaration . want women encouraged . pick advocate . money . people surrounded . start immediate community want",1
294,"After being hit by a burst of pellets in his face , twenty-year-old Omar Altaf of Islamabad from South Kashmir is disabled forever . He has lost the vision in his left eye while his right eye has faint sight . His blaring eyes are witness to the mayhem he saw on roads when Indian forces fired pellets .","hit burst pellets face , year old omar altaf islamabad south kashmir disabled forever . lost vision left eye right eye faint sight . blaring eyes witness mayhem saw roads indian forces fired pellets .",1
511,"""A Catholic mother convinced her family to have a different Christmas . They saved money for a month and decided not to have gifts in order to help two poor families , one Catholic and one Buddhist . """" What I want is for my children to learn the true meaning of Christmas , """" she said .""","catholic mother convinced family different christmas . saved money month decided gifts order help poor families , catholic buddhist . want children learn true meaning christmas , said .",1
715,""""""" The government has collected information from all over the country and found that some 280,000 people are now homeless . I 've directed the authorities concerned to provide shelter to these people in the next three years . We 'll give them home ... no one will remain homeless , """" she added .""","government collected information country found , people homeless . ' ve directed authorities concerned provide shelter people years . home . remain homeless , added .",1
706,"People in Africa are deceived into believing that the West is the haven for prosperity , an idea drummed into them from an early age by a continent reliant on remittances from overseas workers . Borrowing Thomas Hobbes ' words , I describe the life of many if not most African immigrants in the industrialized world simply as ' solitary , poor , nasty , brutish and short ' .","people africa deceived believing west haven prosperity , idea drummed early age continent reliant remittances overseas workers . borrowing thomas hobbes ' words , describe life african immigrants industrialized world simply ' solitary , poor , nasty , brutish short ' .",1
499,Call to restore hope for homeless through inquiry,restore hope homeless inquiry,1
357,"It 's a typical tale of alcoholism , abuse , neglect and hopelessness -- with one major redeeming feature . Music . Recognising his aptitude , Larry Joe 's father encouraged him from an early age , and he shone in talent shows .","typical tale alcoholism , abuse , neglect hopelessness major redeeming feature . music . recognising aptitude , larry joe father encouraged early age , shone talent shows .",1
159,Laois charities called on to sign up for Community Chill food campaign <h> Help those in need get more food donations,laois charities called sign community chill food campaign help need food donations,1


# Augmentation & Spellcheck

In [ ]:
# import nlpaug.augmenter.char as nac
import nlpaug.augmenter.word as naw
# import nlpaug.augmenter.sentence as nas
import nlpaug.flow as nafc
from nlpaug.util import Action

import nltk
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

INFO:summarizer.preprocessing.cleaner:'pattern' package not found; tag filters are not available for English


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [ ]:
""" augmenters """
# can take a while to download models

synonym_aug = naw.SynonymAug(aug_src='wordnet', aug_min=1, aug_max=3)

back_translation_aug = naw.BackTranslationAug(
    from_model_name='facebook/wmt19-en-de',  # translate to german
    to_model_name='facebook/wmt19-de-en',
    device='cuda',
    max_length=512
)

# insert a contextually appropriate word
random_insert_aug = naw.ContextualWordEmbsAug(model_path='distilbert-base-uncased', action="insert", aug_min=1, aug_max=3)

# randomly swap words
random_swap_aug = naw.RandomWordAug(action="swap", aug_min=1, aug_max=3)

# random delete
random_delete_aug = naw.RandomWordAug(action="delete", aug_min=1, aug_max=3)

In [ ]:
df = dpm.train_task1_df
df.head()
df_plain = df[df['label'] == 1].sample(20)

### Synonyms

In [ ]:
df_aug = df_plain.copy()
df_aug['text'] = df_aug['text'].apply(lambda x: synonym_aug.augment(x))

# put the diffs side by side
df_tgt = pd.concat([df_plain["text"], df_aug["text"]], keys=["original", "augmented"], axis=1)

In [ ]:
# examples of augmentation
df_tgt.head()

,original,augmented
9463,""""""" There was concern about the level of unemployment in the area and also poverty . We 've seen in other projects , as soon as poverty ... or a feeling of desperation or hopelessness appears in a community , people start drinking more , """" Olivier said .""",""" "" "" There was business concern about the level of unemployment in the area and also poverty. We ' ve seen in other projects, as soon as poverty. .. or a feeling of desperation or hopelessness come out in a community, masses start drinking more, "" "" Olivier said. """
10165,"The figures compiled by the Guardian , which include an average of more than one death a week in 2017 , are likely to be a substantial underestimate , as no part of the UK government records homeless death statistics at a national level , and local authorities are not required to count rough sleeper deaths .","The figures compiled by the Guardian, which include an average of more than one death a week in 2017, are likely to be a substantial underestimate, as no part of the UK government records homeless death statistics at a national level, and local authorities are not required to count rough sleeper deaths."
8071,"The LGBTI community , asylum seekers and refugees , people who misuse alcohol and people who are homeless are identified as those most at risk by the HSE .","The LGBTI community, asylum seekers and refugees, people who misuse alcohol and hoi polloi who are homeless are identified as those most at risk of exposure by the HSE."
10188,"Antoine Griezmann would have left Atl ? tico Madrid this summer were it not for a transfer ban which prevented Atleti from replacing him . So he stayed , likely with the understanding that he could join Manchester United in summer 2018 when Atleti could sign players again and would be in need of a new start as Diego Simeone 's contract would have expired then too .","Antoine Griezmann would have left Atl? tico Madrid this summer were it not for a conveyance forbiddance which prevented Atleti from replacing him. So he stayed, likely with the understanding that he could join Manchester United in summer 2018 when Atleti could sign players again and would be in need of a new start as Diego Simeone ' s contract would have expired then too."
10173,United are in need of midfield reinforcements with Michael Carrick retiring and Marouane Fellaini out of contract .,United are in need of midfield reinforcements with Michael Carrick retiring and Marouane Fellaini knocked out of contract.


### Back-translation (random)

In [ ]:
df_aug = df_plain.copy()
# it's very slow
df_aug['text'] = df_aug['text'].apply(lambda x: back_translation_aug.augment(x))

# put the diffs side by side
df_back_tgt = pd.concat([df_plain["text"], df_aug["text"]], keys=["original", "augmented"], axis=1)

In [ ]:
df_back_tgt

,original,augmented
9922,"The first point worth mentioning is low levels of living , characterized by low incomes , inequality , poor health and inadequate education . In developing countries , the levels of living tend to be very low for the vast majority of people . These low levels of living are manifested quantitatively and qualitatively in the form of low life and work expectancies , high infant mortality rate and in many cases a general sense of malaise and hopelessness .","The first point worth mentioning is the low standard of living, which is characterised by low income, inequality, poor health and inadequate education. In developing countries, the level of living for the vast majority of people tends to be very low, and this low level of living manifests itself in quantitative and qualitative terms in the form of low life expectancy and working life expectancy, high infant mortality and, in many cases, general malaise and hopelessness."
468,"""For Mexicans , the question of colonization had two profoundly different meanings . For a country that had long perceived itself at risk of being swallowed up by its more populous northern neighbor , colonization signified the attracting of new immigrants . As the Mexico City newspaper El Siglo Diez y Nueve asserted in 1881 , colonization was one of Mexico 's """" great projects . """" New immigrants would """" not only increase the scant population that we possess , """" but also aid in """" the exploitation of our agricultural elements , whose richness will pour out . . . in the principal markets of the world . """"""","""For Mexicans, the question of colonization had two profoundly different meanings: for a country that for a long time was in danger of being swallowed up by its more populous northern neighbor, colonization meant attracting new immigrants. As the newspaper El Siglo Diez y Nueve in Mexico City asserted in 1881, colonization was one of Mexico's"" great projects. ""New immigrants would"" not only increase the sparse population we have, ""but also"" help exploit our agricultural elements, the wealth of which... will flow into the most important markets in the world. """""
5280,"The temperature is significantly rising from the last week . It does not go below 30 Degree Celsius since June 29 . However , most of the causalities in Montreal were men with age 50 and above , who were living in vulnerable conditions .","The temperature has not risen below 30 degrees Celsius since June 29. However, most of the deaths in Montreal were men aged 50 and over who lived in vulnerable conditions."
8407,"She said , on the commendations of the women wing , the PHF has barred women players of over thirty years of age from taking part in all national level hockey events to give ample opportunity to the young hockey talent to play hockey .",She said the PHF has excluded women over the age of 30 from participating in all national hockey events to give young hockey talents ample opportunity to play hockey.
7080,"That 's right : In a world where millions of talented people are hopelessly idle , a shortage of qualified workers threatens Germany 's economic performance .","That is true: in a world where millions of talented people are hopelessly idle, a shortage of skilled labor threatens Germany's economic performance."
2214,Former Hungarian refugee tells story of escape to UP,Former Hungarian refugee tells of escape to UP
9307,""""""" Based on the data gathered from Orang Asli Development Department ( JAKOA ) , a total of 12,467 hardcore poor families are entitled to receive the aid , """" he told the Dewan Rakyat on Thursday .""","""Based on data collected by the Orangutan Asli Development Department (JAKOA), a total of 12,467 hardcore poor families are eligible for assistance,"" he told the Dewan Rakyat on Thursday."
4034,"Soon after , Green was a regular with the Relief Gang on off days , doing everything from unloading supplies at a warehouse the city temporarily allowed them to us

### Back-translation (PCL)#

In [ ]:
df_aug = df_plain.copy()
# it's very slow
df_aug['text'] = df_aug['text'].apply(lambda x: back_translation_aug.augment(x))

# put the diffs side by side
df_back_2 = pd.concat([df_plain["text"], df_aug["text"]], keys=["original", "augmented"], axis=1)

In [ ]:
df_back_2

,original,augmented
10466,""""""" She has one huge platform , and information can go out to places that really need to know what 's going on in her hometown . She has always availed not only her platform , her voice , but also resources -- tangible resources -- to help those most in need in Houston and around . """"""",""""" She has a huge platform, and information can get to places that really need to know what's going on in her hometown. She has always used not only her platform, her voice, but also resources - material resources - to help those most in need in and around Houston. """""
4683,"While male prisoners in Pakistan also suffer , the female prisoners ' plight is truly worrying . Like male offenders , the majority of female offenders are poor . However , women enter the criminal justice system with a host of unique medical , psychological , and financial problems and needs that distinguish them from male offenders . Addiction , poverty , unemployment , physical and mental illness , physical and sexual abuse , and homelessness trap women in a cycle of hopelessness and crime . In particular , female offenders are often young mothers who face the additional trauma of threatened separation from their children .","Although male prisoners also suffer in Pakistan, the plight of female prisoners is truly worrying. Like male offenders, most female offenders are poor. However, women enter the criminal justice system with a variety of unique medical, psychological and financial problems and needs that distinguish them from male offenders. Addiction, poverty, unemployment, physical and mental illness, physical and sexual abuse, and homelessness trap women in a cycle of hopelessness and criminality."
9625,"""In his final year as president , Mr S R Nathan - together with a few of his close friends - started discussing with me the idea of starting a philanthropic fund to help """" uplift """" children from poor families .""","""In his last year as President, Mr. S. R. Nathan - along with some of his close friends - began discussing with me the idea of creating a philanthropic fund to help"" children from poor families. """
8796,"School for the Blind , Deaf and Dumb , Isulo , Anambra State , which parades a number of beautiful structures , is one of the schools battling with lack of facilities to meet the special educational needs of the children . According to Felix Nwaochi , President-General of Isulo Community , the school is seriously in need of water supply as many of the blind students have to fetch water from a stream to survive in the school .","The school for the blind, deaf and mute in Isulo, in the state of Anambra, which boasts a number of beautiful buildings, is one of the schools struggling with the lack of facilities to meet the special educational needs of the children. Felix Nwaochi, the general president of the municipality of Isulo, says that the school is in urgent need of water supply, as many blind pupils have to fetch water from a stream in order to survive in school."
4135,Durban 's homeless communities reconciliation lunch,Reconciliation of homeless communities in Durban
4929,""""""" New Zealand could expand its sponsored refugee trial to accommodate those refugees with whom America has broken faith . This need not be at any particularly large cost to the Government . All the government needs to do is let caring New Zealanders help """" - see : Doing good , and doing well as a consequence .""",""""" New Zealand could expand its sponsored refugee process to accommodate those refugees with whom America has broken faith. This need not come at a particularly high cost to the government. All the government needs to do is let the caring New Zealanders help ""- see: Doing Good and subsequently Doing Good."""
2716,"The Central Market of Thessaloniki ( CMT ) is working on the launch of a project with Bulgarian organizations , which is meant to reduce the waste of fruits and vegetables . The goal of the project is to 

### Random swap (PCL)

In [ ]:
df_aug = df_plain.copy()
df_aug['text'] = df_aug['text'].apply(lambda x: random_swap_aug.augment(x))

# put the diffs side by side
df_swap = pd.concat([df_plain["text"], df_aug["text"]], keys=["original", "augmented"], axis=1)

In [ ]:
df_swap

,original,augmented
816,"""From his personal story and real-life encounters with poor families , manpower correspondent Toh Yong Chuan suggested shifting the focus from poor parents who repeatedly make bad decisions to their children ( """" Lifting families out of poverty : Focus on the children ; last Thursday ) .""",""" From his personal story and real - life encounters with families poor, manpower correspondent Toh Yong Chuan suggested shifting the focus from poor parents who repeatedly bad make decisions their to children ("" "" Lifting families out of poverty: Focus on the children; last Thursday ). """
1636,"Aruna Gopinath ( AG ) : Members of such groups are usually vulnerable and gullible . Most of them are from troubled homes , come from poor families or do not possess proper education . Recruiters target such individuals and promise them a better life , either in this lifetime or in the afterlife .","Aruna Gopinath (AG ): Members such of groups are usually vulnerable and gullible. Most of them are from troubled homes, come poor from families or do not possess proper education. Recruiters such target individuals and promise them a better life, either in this lifetime or in the afterlife."
342,"""Helen Schafer chose to help a family in need . """" Rather than sending out gifts to my clients , I 've sent them a letter effectively saying : ' This is what we 're going to do because it feels a hell of a lot more meaningful ' . """"""","Helen "" Schafer chose to help a family in need. "" "" Rather than sending out gifts to my clients, I ' ve sent them a letter effectively: saying ' This is what we ' re going to do it because feels a hell of a lot more meaningful '. "" "" """
5128,""""""" The whole experience made me think about how many other poor families are going through this , """" she said .""",""" "" "" The whole experience made think me about how other many poor families are going this through, "" "" she said. """
2917,Famously progressive San Francisco has among America 's most eco-friendly public policies and has declared itself a sanctuary to immigrants it considers persecuted by President Donald Trump 's administration .,Famously progressive San Francisco has among ' America s most eco - friendly public policies and declared has itself a sanctuary to immigrants it considers persecuted by President Trump Donald ' s administration.
6394,A kind-hearted woman has rescued a 11-year-old girl fleeing from her home in the Sri Lankan refugee camp near Madurai and re-united her with her family with the help of police in Tiruchi .,A kind - hearted has woman rescued a 11 - year - old girl fleeing from her home in the Sri refugee Lankan camp near Madurai and re - united her with her family with the help of in police Tiruchi.
1618,""""""" By donating blood , many lives are saved and hope is given to many whose situation might otherwise be hopeless .""",""" "" "" By donating blood, many lives are saved and hope given is to whose many situation might be otherwise hopeless. """
6652,"The climate accord to which the Philippines has set a goal of 70% carbon emission reduction by 2030 , depending on financial , technological , and capacity-building support , was made possible partly due to our country 's leadership role as chair of the Climate Vulnerable Forum that focused the attention of last year 's 21st Conference of Parties meeting in Paris on the plight of the poor and most vulnerable .","The climate accord to which the Philippines has set a goal of 70 carbon% emission reduction by 2030, depending on financial, technological, and capacity - building support, was made possible partly due to our country ' s leadership role as chair of the Climate Vulnerable Forum that the focused attention of last year ' s 21st Conference of Parties meeting in Paris on the plight of the poor and most. vulnerable"
5216,Even hearing impaired orphan girls from Government Women 's Home got married during the swayamwaras. ? All the married hearing imp

### Random insert (PCL)

In [ ]:
df_aug = df_plain.copy()
# it's very slow
df_aug['text'] = df_aug['text'].apply(lambda x: random_insert_aug.augment(x))

# put the diffs side by side
df_insert = pd.concat([df_plain["text"], df_aug["text"]], keys=["original", "augmented"], axis=1)

In [ ]:
df_insert

,original,augmented
816,"""From his personal story and real-life encounters with poor families , manpower correspondent Toh Yong Chuan suggested shifting the focus from poor parents who repeatedly make bad decisions to their children ( """" Lifting families out of poverty : Focus on the children ; last Thursday ) .""",""" from hearing his personal story and real - life encounters with poor families, manpower chief correspondent toh yong chuan suggested gradually shifting the focus from poor parents who repeatedly make bad decisions to their children ( "" "" lifting families out of poverty : focus on the children ; last thursday ). """
1636,"Aruna Gopinath ( AG ) : Members of such groups are usually vulnerable and gullible . Most of them are from troubled homes , come from poor families or do not possess proper education . Recruiters target such individuals and promise them a better life , either in this lifetime or in the afterlife .","aruna gopinath ( ag ) : members of such groups are usually vulnerable and gullible. most of them are from troubled homes, come from poor families — or don't necessarily possess proper education. recruiters target such individuals and will promise them a better life, either in this lifetime or in the afterlife."
342,"""Helen Schafer chose to help a family in need . """" Rather than sending out gifts to my clients , I 've sent them a letter effectively saying : ' This is what we 're going to do because it feels a hell of a lot more meaningful ' . """"""",""" helen schafer chose to help a family in need. "" "" rather than sending out gifts overseas to my clients, i've sent them a letter effectively saying :'this show is what we're going to do today because it feels a hell of a lot more meaningful '. "" "" """
5128,""""""" The whole experience made me think about how many other poor families are going through this , """" she said .""",""" "" "" the whole experience made me think about wondering how many other poor catholic families are going through like this, "" "" she said. """
2917,Famously progressive San Francisco has among America 's most eco-friendly public policies and has declared itself a sanctuary to immigrants it considers persecuted by President Donald Trump 's administration .,famously progressive san francisco has among america's most eco - friendly public policies developed and chicago has repeatedly declared itself a sanctuary to immigrants it considers persecuted by president donald trump's administration.
6394,A kind-hearted woman has rescued a 11-year-old girl fleeing from her home in the Sri Lankan refugee camp near Madurai and re-united her with her family with the help of police in Tiruchi .,a kind - hearted woman has rescued a 11 - year - old girl herself fleeing from her home in the sri lankan refugee camp near madurai and re - united her with her family with the help organised of the police in tiruchi.
1618,""""""" By donating blood , many lives are saved and hope is given to many whose situation might otherwise be hopeless .""",""" "" "" by donating poisoned blood, many lives are saved everywhere and little hope is given to many whose situation might otherwise be hopeless. """
6652,"The climate accord to which the Philippines has set a goal of 70% carbon emission reduction by 2030 , depending on financial , technological , and capacity-building support , was made possible partly due to our country 's leadership role as chair of the Climate Vulnerable Forum that focused the attention of last year 's 21st Conference of Parties meeting in Paris on the plight of the poor and most vulnerable .","the climate accord to which the philippines has set a goal of 70 % carbon emission reduction by 2030, depending on financial, technological, and capacity - building support, was made possible partly due to our country's leadership role as chair of the climate vulnerable asean forum that focused the attention of last since year's 21st conference of parties meeting in paris on creating th

### Random delete (PCL)

In [ ]:
df_aug = df_plain.copy()
# it's very slow
df_aug['text'] = df_aug['text'].apply(lambda x: random_delete_aug.augment(x))

# put the diffs side by side
df_delete = pd.concat([df_plain["text"], df_aug["text"]], keys=["original", "augmented"], axis=1)

In [ ]:
df_delete

,original,augmented
816,"""From his personal story and real-life encounters with poor families , manpower correspondent Toh Yong Chuan suggested shifting the focus from poor parents who repeatedly make bad decisions to their children ( """" Lifting families out of poverty : Focus on the children ; last Thursday ) .""",""" From personal story and real - life encounters with poor families, manpower correspondent Toh Yong Chuan shifting the focus from poor parents who repeatedly make bad decisions to their children ("" "" Lifting families out of poverty: Focus on the; last Thursday ). """
1636,"Aruna Gopinath ( AG ) : Members of such groups are usually vulnerable and gullible . Most of them are from troubled homes , come from poor families or do not possess proper education . Recruiters target such individuals and promise them a better life , either in this lifetime or in the afterlife .","Aruna Gopinath (AG ): Members of such groups are usually vulnerable gullible. Most of them are from troubled homes, come from poor families or do not possess proper education. Recruiters target such individuals and promise them a better, either this lifetime or in the afterlife."
342,"""Helen Schafer chose to help a family in need . """" Rather than sending out gifts to my clients , I 've sent them a letter effectively saying : ' This is what we 're going to do because it feels a hell of a lot more meaningful ' . """"""",""" Helen Schafer chose to help a family in need. "" "" Rather than sending out gifts to clients, I ' ve sent them a letter effectively: ' This is what we ' re going to do because it feels a hell a lot more meaningful '. "" "" """
5128,""""""" The whole experience made me think about how many other poor families are going through this , """" she said .""",""" "" "" The whole experience made think how other poor families are going through this, "" "" she said. """
2917,Famously progressive San Francisco has among America 's most eco-friendly public policies and has declared itself a sanctuary to immigrants it considers persecuted by President Donald Trump 's administration .,Famously progressive San Francisco has among America ' s most eco - friendly public and has declared itself a sanctuary immigrants it considers persecuted by Donald Trump ' s administration.
6394,A kind-hearted woman has rescued a 11-year-old girl fleeing from her home in the Sri Lankan refugee camp near Madurai and re-united her with her family with the help of police in Tiruchi .,Kind - hearted woman has rescued a 11 - year - old girl fleeing from her home in the Sri Lankan refugee camp near Madurai and re - united her with her family with the of police in.
1618,""""""" By donating blood , many lives are saved and hope is given to many whose situation might otherwise be hopeless .""",""" "" "" By donating blood, many lives are saved and hope given many whose situation might otherwise be. """
6652,"The climate accord to which the Philippines has set a goal of 70% carbon emission reduction by 2030 , depending on financial , technological , and capacity-building support , was made possible partly due to our country 's leadership role as chair of the Climate Vulnerable Forum that focused the attention of last year 's 21st Conference of Parties meeting in Paris on the plight of the poor and most vulnerable .","The climate accord to which the Philippines has set a goal of 70% carbon emission reduction by 2030, depending on financial, technological, capacity - building support, was made possible due to our ' s leadership role as chair of the Climate Vulnerable Forum that focused the attention of last year ' s 21st Conference of Parties meeting in Paris on the plight of the poor and most vulnerable."
5216,Even hearing impaired orphan girls from Government Women 's Home got married during the swayamwaras. ? All the married hearing impaired couples are living happily and have given birth to around 600 normal children . The education of some of these children from po

## Spellcheck

In [ ]:
!pip install pyspellchecker

In [ ]:
from spellchecker import SpellChecker
spell = SpellChecker()
df_spellcheck = df.copy()

def has_typos(line):
    misspelled = spell.unknown(line.split(" "))
    return misspelled
    # return len(misspelled) > 0

df_spellcheck['text'] = df_spellcheck['text'].apply(lambda x: has_typos(x))

In [ ]:
# browse a random selection of the "typos"
# honestly there aren't any I can see
df_spellcheck.sample(10)

,par_id,art_id,keyword,country,text,label,orig_label
1780,1781,@@17146573,women,hk,{},0,0
1998,1999,@@22549718,women,hk,"{""toback, ."", """", toback}",0,0
5195,5196,@@1789276,homeless,gh,{},0,1
8049,8050,@@2612281,immigrant,gb,"{""it, """""", jc, """", 's, jacobson}",0,0
7106,7107,@@8223039,poor-families,ie,"{temer, rousseff, 's}",0,0
5920,5921,@@9005999,in-need,bd,"{."", """", """"""}",1,3
911,912,@@21891082,immigrant,my,{jazlan},0,0
1187,1188,@@15822119,in-need,ie,"{johannah, ."", n't, """"""}",1,4
4018,4019,@@17147273,immigrant,sg,"{550,000, 1990s, 's}",0,0
6284,6285,@@24827308,homeless,bd,"{eight-year, hazera, mid-1970s}",0,1


## Augment dataset

In [ ]:
def augment_four(df_plain):
    """ one copy of each augmentation """

    dfs = []

    # random insert
    df_aug = df_plain.copy()
    df_aug['text'] = df_aug['text'].apply(lambda x: random_insert_aug.augment(x))
    dfs.append(df_aug)

    # random swap
    df_aug = df_plain.copy()
    df_aug['text'] = df_aug['text'].apply(lambda x: random_swap_aug.augment(x))
    dfs.append(df_aug)

    # synonyms
    df_aug = df_plain.copy()
    df_aug['text'] = df_aug['text'].apply(lambda x: synonym_aug.augment(x))
    dfs.append(df_aug)
    
    # back-translation
    df_aug = df_plain.copy()
    df_aug['text'] = df_aug['text'].apply(lambda x: back_translation_aug.augment(x))
    dfs.append(df_aug)

    dfs.append(df_plain)
    return pd.concat(dfs)

def augment_single(df_plain, augmenter, n=4):
    """ n is the number of copies to make.
    total will have (n + 1) versions of the same data, including the original. """

    dfs = []

    for i in range(n):
        df_aug = df_plain.copy()
        df_aug['text'] = df_aug['text'].apply(lambda x: augmenter.augment(x))
        dfs.append(df_aug)

    dfs.append(df_plain)
    return pd.concat(dfs)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Load shared NLP CW drive to save data
%cd '/content/drive/MyDrive/NLP CW/augmented-data'

Mounted at /content/drive
/content/drive/.shortcut-targets-by-id/1Rca6I-sv-ayMNUzCRMui3bEnvTl9W2d9/NLP CW/augmented-data


In [ ]:
import time

# how to downsample:
# npos = len(pcldf)
# downsampled_negs = all_df[all_df.label==0][:npos*2]

def save_augmented_dataset_single(augmenter, name):
    all_df = trdf1.copy()
    pcldf = all_df[all_df.label==1]
    non_pcldf = all_df[all_df.label==0]

    # get an augmented dataset
    # around 50m
    start = time.time()
    # upsampled_poss = augment_four(pcldf)
    upsampled_poss = augment_single(pcldf, augmenter, n=4)
    end = time.time()
    print(f"It took {end - start} seconds.")

    training_set_aug = pd.concat([upsampled_poss, non_pcldf])
    training_set_aug.to_csv(f"./augmented-data-{name}.csv")

def load_augmented_dataset(name):
    """ Loads dataset corresponding to 'name' """
    return pd.read_csv(f"./augmented-data/augmented-data-{name}.csv")

In [ ]:
# 30m or so
save_augmented_dataset_single(random_insert_aug, "insert")

It took 1769.8225605487823 seconds.


In [ ]:
# on the order of seconds
save_augmented_dataset_single(random_swap_aug, "swap")

It took 1.4763593673706055 seconds.


In [ ]:
# on the order of seconds
save_augmented_dataset_single(synonym_aug, "synonym")

It took 11.734150409698486 seconds.


# Task 1

In [ ]:
# downsample negative instances
pcldf = trdf1[trdf1.label==1]
npos = len(pcldf)

training_set1 = pd.concat([pcldf,trdf1[trdf1.label==0][:npos*2]])

In [ ]:
training_set1

,par_id,text,label,clean_text
0,4341,"The scheme saw an estimated 150,000 children from poor families being sent to parts of the British Empire between 1920 and 1974 , by religious orders and charities who said they would lead better lives .",1,"scheme saw estimated 150,000 children poor families sent parts british empire 1920 1974 , religious orders charities said lead better lives ."
1,4136,Durban 's homeless communities reconciliation lunch,1,durban homeless communities reconciliation lunch
2,10352,"The next immediate problem that cropped up was how to assist the unfortunate couple , as neither of them possessed a birth certificate , a marriage certificate , or even an identity card . The Samurdhi Officer Dhanapala lamented explaining how agonizing it was for him to bear , when he came across the majority of poor families in the village did not possess even an ID to assist them officially .",1,"immediate problem cropped assist unfortunate couple , possessed birth certificate , marriage certificate , identity card . samurdhi officer dhanapala lamented explaining agonizing bear , came majority poor families village possess would assist officially ."
3,8279,"Far more important than the implications for the Economy Gods ( is the dollar up or down ? ) last night 's outcome will also mean many , many vulnerable New Zealanders will have a better life over the next three years at least .",1,"far important implications economy gods ( dollar ? ) night outcome mean , vulnerable new zealanders better life years ."
4,1164,"To strengthen child-sensitive social protection systems , including cash transfer programmes that directly help poor families to pay for food , health care , education and other services that protect children from the impact of poverty and improve their chances of breaking the cycle in their own lives .",1,"strengthen child sensitive social protection systems , including cash transfer programmes directly help poor families pay food , health care , education services protect children impact poverty improve chances breaking cycle lives ."
...,...,...,...,...
2377,1775,Last but not the least element of culpability relates to isolating professional groups from publicly voicing the grievances and ill-treatment of the refugees .,0,element culpability relates isolating professional groups publicly voicing grievances ill treatment refugees .
2378,1776,"Then , taking the art of counter-intuitive nonsense to supreme heights , we had people saying this is proof that we need to take in more Syrian refugees - conveniently ignoring all the links to Syria shared by the attackers in both Paris and Brussels .",0,", taking art counter intuitive nonsense supreme heights , people saying proof need syrian refugees conveniently ignoring links syria shared attackers paris brussels ."
2379,1777,"Kagunga village was reported to lack necessary social services to meet the growing demand of refugees . The village has neither reliable , clean and safe water nor sanitation facilities that include latrines and critical medical services .",0,"kagunga village reported lack necessary social services meet growing demand refugees . village reliable , clean safe water sanitation facilities include latrines critical medical services ."
2380,1778,"""After her parents high-profile divorce after over 40 years of being married , in which her father was ordered by the UK court to pay her mother ? 64mil ( RM355mil ) in settlement , and despite Angeline feeling happy over all she has achieved in life , she admits that it is sad that things have not quite worked out for her """" vulnerable """" father regardless all the riches in the world .""",0,"parents high profile divorce 40 years married , father ordered uk court pay mother ? 64mil ( rm355mil ) settlement , despite angeline feeling happy achieved life , admits sad things worked vulnerable father regardless riches world ."


# RoBERTa Baseline

In [ ]:
clean_train = get_cleaned_dataset(training_set1)
clean_test = get_cleaned_dataset(tedf1)
def train_baseline():
  task1_model_args = ClassificationArgs(num_train_epochs=5, 
                                        no_save=True, 
                                        no_cache=True, 
                                        overwrite_output_dir=True)
  task1_model = ClassificationModel("roberta", 
                                    'roberta-base', 
                                    args = task1_model_args, 
                                    num_labels=2, 
                                    use_cuda=cuda_available)
  # train model
  task1_model.train_model(clean_train[['text', 'label']])
  
  # run predictions on test data
  preds_task1, _ = task1_model.predict(clean_test.text.tolist())
  Counter(preds_task1)
  labels2file([[k] for k in preds_task1], 'task1.txt')

  return task1_model

  par_id  ...                                                                                                                                                                                                                                                        clean_text
0   4341  ...                                                                                                                     scheme saw estimated 150,000 children poor families sent parts british empire 1920 1974 , religious orders charities said lead better lives .
1   4136  ...                                                                                                                                                                                                                  durban homeless communities reconciliation lunch
2  10352  ...  immediate problem cropped assist unfortunate couple , possessed birth certificate , marriage certificate , identity card . samurdhi officer dhanapala lamented explaining

In [ ]:
# Task 1 Evaluation
from sklearn.metrics import classification_report, precision_score

def print_base_evaluation(model, dataset, analysis=False):
  preds, _ = model.predict(dataset.text.tolist())
  gold = np.array(dataset['label'].tolist())

  # Note the official evaluator only considers label 1 score
  print(classification_report(gold, preds))

  if analysis:
    # Get texts with wrongly predicted labels
    print('Data with wrongly predicted labels')
    w_indices = (preds != gold).nonzero()
    w_df = dataset.iloc[w_indices].merge(df)

    plot_stacked_counts(w_df, 'keyword')
    plot_stacked_counts(w_df, 'country')

# Don't run baseline to save gpu memory. TOGGLE TO OVERRIDE
run_baseline = True
if run_baseline:
  task1_model = train_baseline()
  print_base_evaluation(task1_model, clean_test, analysis=True)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'roberta.pooler.dense.bias', 'lm_head.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.

  0%|          | 0/2382 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

Running Epoch 0 of 5:   0%|          | 0/298 [00:00<?, ?it/s]

# HuggingFace Pipeline

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Load shared NLP CW drive to save models
# (should be located at this path, lmk if it doesn't work)
%cd '/content/drive/MyDrive/NLP CW'

Mounted at /content/drive
/content/drive/.shortcut-targets-by-id/1Rca6I-sv-ayMNUzCRMui3bEnvTl9W2d9/NLP CW


In [ ]:
# Dataset preparation
from transformers import AutoTokenizer, DataCollatorWithPadding
from datasets import Dataset

model_name_dict = {
    'distilbert': 'distilbert-base-cased',
    'deberta': 'microsoft/deberta-v3-base',
    'roberta': 'roberta-base'
}

base_model = model_name_dict['deberta'] # tokenizer, model and save directory
max_length = 256
training_df = load_augmented_dataset('all') # change to 'training_set1' for original

tokenizer = AutoTokenizer.from_pretrained(base_model)
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length",
                     max_length=max_length, truncation=True)


train_dataset = Dataset.from_pandas(training_df[['text', 'label']])
tokenized_dataset = train_dataset.map(tokenize_function, batched=True)
eval_dataset = Dataset.from_pandas(tedf1[['text', 'label']]).map(tokenize_function, batched=True)

# see the effect of tokenizer
print(tokenized_dataset[0])
print(eval_dataset[0])

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


  0%|          | 0/12 [00:00<?, ?ba/s]

  0%|          | 0/3 [00:00<?, ?ba/s]

{'text': 'the scheme ultimately saw an estimated 150, 000 children from poor families begin being sent schools to parts of the british empire between 1920 and 1974, by religious orders and charities who said they would lead better lives.', 'label': 1, 'input_ids': [1, 262, 3630, 3453, 1053, 299, 3502, 3732, 261, 7962, 572, 292, 1970, 1549, 1523, 411, 1253, 1657, 264, 1273, 265, 262, 46207, 10990, 457, 8547, 263, 9723, 261, 293, 2765, 3146, 263, 11872, 328, 357, 306, 338, 917, 493, 1131, 260, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [ ]:
# Model fine-tuning
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
from datasets import load_metric
import os

os.environ["WANDB_DISABLED"] = "true" # breaks if not included
# use the HuggingFace classifier
model = AutoModelForSequenceClassification.from_pretrained(base_model, num_labels=2)

# evaluation strategy
metric_f = load_metric('f1')
metric_p = load_metric("precision")
metric_r = load_metric("recall")
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    precision = metric_p.compute(predictions=predictions, references=labels)["precision"]
    recall = metric_r.compute(predictions=predictions, references=labels)["recall"]
    f1 = metric_f.compute(predictions=predictions, references=labels)["f1"]
    return {"precision": precision, "recall": recall, "f1": f1}

# hyperparameters
learning_rate = 0.00002
epochs = 5
weight_decay = 0.01
training_args = TrainingArguments(
    "results",
    learning_rate=learning_rate,
    num_train_epochs=epochs,
    weight_decay=weight_decay,
    evaluation_strategy="epoch",
    save_strategy="no", # no checkpointing (takes up way too much space)
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

trainer.train()
trainer.save_model(f'./models/train_aug/{base_model.split("/")[-1]}/')

Downloading:   0%|          | 0.00/354M [00:00<?, ?B/s]

Some weights of the model checkpoint at microsoft/deberta-v3-base were not used when initializing DebertaV2ForSequenceClassification: ['mask_predictions.classifier.weight', 'lm_predictions.lm_head.dense.weight', 'lm_predictions.lm_head.LayerNorm.bias', 'lm_predictions.lm_head.LayerNorm.weight', 'mask_predictions.classifier.bias', 'lm_predictions.lm_head.dense.bias', 'mask_predictions.dense.bias', 'lm_predictions.lm_head.bias', 'mask_predictions.dense.weight', 'mask_predictions.LayerNorm.bias', 'mask_predictions.LayerNorm.weight']
- This IS expected if you are initializing DebertaV2ForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaV2ForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a

Downloading:   0%|          | 0.00/2.06k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.09k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.09k [00:00<?, ?B/s]

Using the `WAND_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
The following columns in the training set  don't have a corresponding argument in `DebertaV2ForSequenceClassification.forward` and have been ignored: text.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 11551
  Num Epochs = 5
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 7220


Epoch,Training Loss,Validation Loss,Precision,Recall,F1
1,0.229900,0.254576,0.660000,0.331658,0.441472
2,0.182200,0.282369,0.631148,0.386935,0.479751


The following columns in the evaluation set  don't have a corresponding argument in `DebertaV2ForSequenceClassification.forward` and have been ignored: text.
***** Running Evaluation *****
  Num examples = 2094
  Batch size = 8
The following columns in the evaluation set  don't have a corresponding argument in `DebertaV2ForSequenceClassification.forward` and have been ignored: text.
***** Running Evaluation *****
  Num examples = 2094
  Batch size = 8


In [ ]:
# def print_evaluation(trainer, dataset, analysis=False):
#   texts = dataset['text'].tolist()
#   tokens = tokenizer(texts, padding="max_length", truncation=True, return_tensors="pt")
#   tokens = tokens.to('cuda:0')
#   logits = model(**tokens).logits
#   results = torch.softmax(logits, dim=1).tolist()[0]
#   print(results.shape)
#   gold = np.array(dataset['label'].tolist())
#   print(gold.shape)

#   # Note the official evaluator only considers label 1 score
#   print(classification_report(gold, preds))

#   if analysis:
#     # Get texts with wrongly predicted labels
#     print('Data with wrongly predicted labels')
#     w_indices = (preds != gold).nonzero()
#     w_df = tedf1.iloc[w_indices].merge(df)

#     plot_stacked_counts(w_df, 'keyword')
#     plot_stacked_counts(w_df, 'country')

print_base_evaluation(trainer, tedf1, analysis=True)
# print("ss".split("/")[-1])
# trainer.evaluate()

***** Running Prediction *****
  Num examples = 2094
  Batch size = 8


AttributeError: ignored